In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import feather

PID = 'gasco-analytics'

In [3]:
query = '''SELECT *
FROM
(SELECT * FROM (
SELECT telefono,
max(mobile) mobile,
max(customerNoIfOffline) customerNoIfOffline,
SUM(IF(EXTRACT(DAYOFWEEK FROM createdat) = 1,1,0)) compras_domingo,
SUM(IF(EXTRACT(DAYOFWEEK FROM createdat) = 2,1,0)) compras_lunes,
SUM(IF(EXTRACT(DAYOFWEEK FROM createdat) = 3,1,0)) compras_martes,
SUM(IF(EXTRACT(DAYOFWEEK FROM createdat) = 4,1,0)) compras_miercoles,
SUM(IF(EXTRACT(DAYOFWEEK FROM createdat) = 5,1,0)) compras_jueves,
SUM(IF(EXTRACT(DAYOFWEEK FROM createdat) = 6,1,0)) compras_viernes,
SUM(IF(EXTRACT(DAYOFWEEK FROM createdat) = 7,1,0)) compras_sabado,
SUM(IF(EXTRACT(MONTH FROM createdat) IN (12,1,2),1,0)) compras_verano,
SUM(IF(EXTRACT(MONTH FROM createdat) IN (3,4,5),1,0)) compras_otonio,
SUM(IF(EXTRACT(MONTH FROM createdat) IN (6,7,8),1,0)) compras_invierno,
SUM(IF(EXTRACT(MONTH FROM createdat) IN (9,10,11),1,0)) compras_primavera,
TRIM(STRING_AGG(DISTINCT(IF(orden_pedidos_clientes=1,sector,'')),'')) comuna, 
COUNT(DISTINCT(address)) distintas_direcciones,
IF(SUM(IF(address LIKE '%depto%' OR address LIKE '%dpto%' OR address LIKE '%departamento%' OR address LIKE '%dto%' OR address LIKE '%dp%' OR address LIKE '%dppt%' OR address LIKE '%dpt%' OR address LIKE '%edificio%' , 1, 0 ))>0,1,0)  depto_casa,
CASE
WHEN min(rango_edad)= 'Menor 24' THEN 1
WHEN min(rango_edad) = '25 - 34' THEN 2
WHEN min(rango_edad) = '35 - 44' THEN 3
WHEN min(rango_edad) = '45 - 54' THEN 4
WHEN min(rango_edad) = '55 - 64' THEN 5
WHEN min(rango_edad) = 'Mayor 65' THEN 6
ELSE null
END edad,
max(genero) sexo,
COUNT(*) cantidad_pedidos,
SUM(totalAmount_GC) suma_pedido, 
SUM(discountAmount) suma_descuentos, --hay que sumar los cupones? cambiar esta variable? Tomar alguna variable de cupones o campañas?.. Hacer CASE con online/offline
SUM(total_kilos_pedido) suma_kilos,
SUM(p_CN02) suma_p_CN02,
SUM(p_CN05)+SUM(p_CS05) suma_formato_05, 
SUM(p_CN11)+SUM(p_CS11) suma_formato_11, 
SUM(p_CN15)+SUM(p_CS15) suma_formato_15, 
SUM(p_CN45)+SUM(p_CS45) suma_formato_45, 
SUM(p_CGH15) suma_p_CGH15, 
SUM(p_CGH15_AL) suma_p_CGH15_AL, 
SUM(p_GGH15_AL) suma_p_GGH15_AL,
ROUND(AVG(frecuencia),1) frec_compra_promedio_dias,
IF(SUM(IF(tipo_pedido_bien = 'OFFLINE',1,0))>0,1,0) canal_offline,
IF(SUM(IF(tipo_pedido_bien = 'ONLINE',1,0))>0,1,0) canal_online,
IF(SUM(IF(tipo_pedido_bien NOT IN ('ONLINE','OFFLINE'),1,0))>0,1,0) otro_canal,
IF(SUM(IF(paymentMethodDescription = 'Efectivo',1,0))>0,1,0) pago_efectivo,
IF(SUM(IF(paymentMethodDescription = 'Tarjeta Debito/Credito',1,0))>0,1,0) pago_deb_cred,
IF(SUM(IF(paymentMethodDescription = 'CuentaRUT/Ctas BancoEstado',1,0))>0,1,0) pago_bco_estado,
IF(SUM(IF(paymentMethodDescription NOT IN ('CuentaRUT/Ctas BancoEstado','Tarjeta Debito/Credito','Efectivo'),1,0))>0,1,0) pago_otros
FROM 
(SELECT *,
DATE_DIFF(EXTRACT(DATE FROM createdat),LAG(EXTRACT(DATE FROM createdat)) OVER(PARTITION BY telefono ORDER BY EXTRACT(DATE FROM createdat)), DAY) frecuencia,
row_number() OVER(PARTITION BY telefono ORDER BY createdat DESC) orden_pedidos_clientes,
FROM
(select *,
CASE 
WHEN tipo_pedido_bien = 'OFFLINE' THEN customerNoIfOffline
WHEN tipo_pedido_bien = 'ONLINE' THEN mobile
ELSE null
END AS telefono FROM `gasco.pedidos_gasconnect_todos_javier`)) 
GROUP BY telefono
)
WHERE mobile NOT IN ('949700781','987446282','946272711','959855281','937198925','982834413','940320641','933486794','968495840','982169639','976719792','976837510','946900115','952311351','966376800','938728120','920165243','937181726','936105770','999755969','956037246','971393407','964272568','996937599','958284792','944210340','972379273','931776921','981218376','930550981','938762481','949152671','989143697','969034005','986498191','920767324','944088616','975301909','920031082','994060817','965892521','953211031','956111329','995696387','966447842','996461327','997936889','933569506','933763374','936831280','985518824','973616165','959886604','965845250','973905440','956945238','965847403','954433370','957660478','948104628','966148636','967120855','947844436','969027661','959624405','977497046','950208898','957999693','989708139','986711311','982651844','952101141','962320325','933396681','948703345','972840071','981959179','975933937','945779141','945779142','937411889','978556265','951886537','932592671','964435798','987149470','920596293','987110816','979505347','935543084','996129257','954838361','959938922','950888056','936831280','954653898','965838210','965688521','959865600','956654007','942281547','965296076','972045344','968281852','942399781','992708206','952564809','976299318','997862235','950725383','930731503','944906499','931830297','935544746','966294876','989255095','994842012','966996365','978994717','978805734','965265205','942556019','935544746','967514899','950208898','968429653','940312619','987492429','963868077','994497785','965937013','948116910','986757302','945168176','973772861','954423660','987070184','950207273','968495840','976100365','991675804','973345579','946274646','995994648','982808794','931238125','985326354','930910440','981990488','955304961','966863150','978257363','935224124','984114608','952240390','932957108','938866484','947906680','990573502','950384188','962787325','995476328','932598533','935544746','920300776','977067735','941186610','945765224','997250292','941606975','950738096','920610205','998853511','993245736','920315263','964814674','982507874','956678416','920282559','978225968','945797466','995766773','987268438','991409989','968665725','920394138','935157070','978187654','964451113','930319282','940521015','936547434','971023190','966974624','936892997','989203774','935544746','957942982','987212671','963518304','954433370','940017083','973551964','961988683','977663841','972585152','946589725','964779674','984419260','997626843','941040263','937068078','972398188','940884370','944685598','937056940','949360373','988991779','949275692','946158245','968034996','961412983','986701122','958475876','998250053') 
OR customerNoifOffline NOT IN ('949700781','987446282','946272711','959855281','937198925','982834413','940320641','933486794','968495840','982169639','976719792','976837510','946900115','952311351','966376800','938728120','920165243','937181726','936105770','999755969','956037246','971393407','964272568','996937599','958284792','944210340','972379273','931776921','981218376','930550981','938762481','949152671','989143697','969034005','986498191','920767324','944088616','975301909','920031082','994060817','965892521','953211031','956111329','995696387','966447842','996461327','997936889','933569506','933763374','936831280','985518824','973616165','959886604','965845250','973905440','956945238','965847403','954433370','957660478','948104628','966148636','967120855','947844436','969027661','959624405','977497046','950208898','957999693','989708139','986711311','982651844','952101141','962320325','933396681','948703345','972840071','981959179','975933937','945779141','945779142','937411889','978556265','951886537','932592671','964435798','987149470','920596293','987110816','979505347','935543084','996129257','954838361','959938922','950888056','936831280','954653898','965838210','965688521','959865600','956654007','942281547','965296076','972045344','968281852','942399781','992708206','952564809','976299318','997862235','950725383','930731503','944906499','931830297','935544746','966294876','989255095','994842012','966996365','978994717','978805734','965265205','942556019','935544746','967514899','950208898','968429653','940312619','987492429','963868077','994497785','965937013','948116910','986757302','945168176','973772861','954423660','987070184','950207273','968495840','976100365','991675804','973345579','946274646','995994648','982808794','931238125','985326354','930910440','981990488','955304961','966863150','978257363','935224124','984114608','952240390','932957108','938866484','947906680','990573502','950384188','962787325','995476328','932598533','935544746','920300776','977067735','941186610','945765224','997250292','941606975','950738096','920610205','998853511','993245736','920315263','964814674','982507874','956678416','920282559','978225968','945797466','995766773','987268438','991409989','968665725','920394138','935157070','978187654','964451113','930319282','940521015','936547434','971023190','966974624','936892997','989203774','935544746','957942982','987212671','963518304','954433370','940017083','973551964','961988683','977663841','972585152','946589725','964779674','984419260','997626843','941040263','937068078','972398188','940884370','944685598','937056940','949360373','988991779','949275692','946158245','968034996','961412983','986701122','958475876','998250053')
)'''

df = pd.read_gbq(query=query, project_id=PID, dialect="standard", use_bqstorage_api=True) 

In [4]:
df.head()

,telefono,mobile,customerNoIfOffline,compras_domingo,compras_lunes,compras_martes,compras_miercoles,compras_jueves,compras_viernes,compras_sabado,...,suma_p_CGH15_AL,suma_p_GGH15_AL,frec_compra_promedio_dias,canal_offline,canal_online,otro_canal,pago_efectivo,pago_deb_cred,pago_bco_estado,pago_otros
0,+56-9-000063643272,0000000000,+56-9-000063643272,0,1,0,0,0,0,0,...,0.0,0.0,NaN,1,0,0,1,0,0,0
1,10059328,569000087914890,10059328,1,0,0,0,0,1,1,...,0.0,0.0,101.0,1,0,0,1,0,0,0
2,225279038,225279038,225279038,0,2,0,1,0,0,0,...,0.0,0.0,79.5,1,0,0,1,0,0,0
3,225313432,0000000000,225313432,0,0,0,0,0,1,0,...,0.0,0.0,NaN,1,0,0,1,0,0,0
4,225321546,0000000000,225321546,0,2,3,2,2,2,3,...,0.0,0.0,43.3,1,0,0,1,0,0,0


In [5]:
numeric = ['compras_domingo',
 'compras_lunes',
 'compras_martes',
 'compras_miercoles',
 'compras_jueves',
 'compras_viernes',
 'compras_sabado',
 'compras_verano',
 'compras_otonio',
 'compras_invierno',
 'compras_primavera',
 'cantidad_pedidos',
 'edad',
 'suma_pedido',
 'suma_descuentos',
 'suma_kilos',
 'suma_p_CN02',
 'suma_formato_05',
 'suma_formato_11',
 'suma_formato_15',
 'suma_formato_45',
 'suma_p_CGH15',
 'suma_p_CGH15_AL',
 'suma_p_GGH15_AL',
 'frec_compra_promedio_dias']

categorical = ['comuna',
 'depto_casa',
 'sexo',
 'canal_offline',
 'canal_online',
 'otro_canal',
 'pago_efectivo',
 'pago_deb_cred',
 'pago_bco_estado',
 'pago_otros']

#No considerar
otras = ['telefono',
 'mobile',
 'customerNoIfOffline',
 'distintas_direcciones']

In [6]:
import h2o
from h2o.automl import H2OAutoML

In [7]:
features = numeric + categorical
H2O_server = h2o.init(port=54321, min_mem_size='15g')
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,3 hours 7 mins
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.9
H2O cluster version age:,"1 year, 3 months and 20 days !!!"
H2O cluster name:,H2O_from_python_ubuntu_jabhe4
H2O cluster total nodes:,1
H2O cluster free memory:,13.76 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [8]:
nombre_modelo_calefont = '/home/ubuntu/spike/Emi/modelo_uso_gas/202102/GBM_grid_1_AutoML_20210217_145227_model_23'

#Anterior: XGBoost_grid_1_AutoML_20200909_163641_model_6

In [9]:
column_types = {key: 'enum' for key in categorical}
column_types.update({key: 'numeric' for key in numeric})

In [10]:
df_h2o_test_calefont = h2o.H2OFrame(df, destination_frame="df_para_pred",
                          column_types=column_types)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [11]:
#Load model, predecir y merge
modelo_calefont = h2o.load_model(nombre_modelo_calefont)
preds_calefont = modelo_calefont.predict(df_h2o_test_calefont)
preds_y_datos = pd.concat([df_h2o_test_calefont.as_data_frame(), preds_calefont.as_data_frame()], axis=1)

gbm prediction progress: |████████████████████████████████████████████████| 100%


/home/ubuntu/miniconda3/envs/spike_basicoV5/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'comuna' has levels not trained on: [RM   REGIÓN METROPOLITANA, XII  REGIÓN DE MAGALLANES Y LA ANTÁRTICA CHILENA]
  warnings.warn(w)


In [12]:
nombre_modelo_cocina = '/home/ubuntu/spike/Emi/modelo_uso_gas/202102/XGBoost_grid_1_AutoML_20210217_181338_model_6'

#Anterior: XGBoost_grid_1_AutoML_20200909_135120_model_7

In [13]:
df_h2o_test_cocina = h2o.H2OFrame(df, destination_frame="df_para_pred_cocina",
                          column_types=column_types)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [14]:
#Load model, predecir y merge
modelo_cocina = h2o.load_model(nombre_modelo_cocina)
preds_cocina = modelo_cocina.predict(df_h2o_test_cocina)
preds_y_datos_2 = pd.concat([preds_y_datos, preds_cocina.as_data_frame()], axis=1)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [15]:
nombre_modelo_estufa = '/home/ubuntu/spike/Emi/modelo_uso_gas/202102/XGBoost_grid_1_AutoML_20210217_195245_model_6'

#/home/ubuntu/spike/Emi/modelo_uso_gas/XGBoost_grid_1_AutoML_20200909_163519_model_1

In [16]:
df_h2o_test_estufa = h2o.H2OFrame(df, destination_frame="df_para_pred_estufa",
                          column_types=column_types)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [17]:
#Load model, predecir y merge
modelo_estufa = h2o.load_model(nombre_modelo_estufa)
preds_estufa = modelo_estufa.predict(df_h2o_test_estufa)
preds_y_datos_3 = pd.concat([preds_y_datos_2, preds_estufa.as_data_frame()], axis=1)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [18]:
nombre_modelo_parrilla = '/home/ubuntu/spike/Emi/modelo_uso_gas/202102/XGBoost_grid_1_AutoML_20210218_125943_model_7'

#Anterior: XGBoost_grid_1_AutoML_20200909_134948_model_10

In [19]:
df_h2o_test_parrilla = h2o.H2OFrame(df, destination_frame="df_para_pred_parrilla",
                          column_types=column_types)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [20]:
#Load model, predecir y merge
modelo_parrilla = h2o.load_model(nombre_modelo_parrilla)
preds_parrilla = modelo_parrilla.predict(df_h2o_test_parrilla)
preds_y_datos_4 = pd.concat([preds_y_datos_3, preds_parrilla.as_data_frame()], axis=1)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [21]:
preds_y_datos_4.columns.values[39] = "predict_calefont"
preds_y_datos_4.columns.values[40] = "p0_calefont"
preds_y_datos_4.columns.values[41] = "p1_calefont"
preds_y_datos_4.columns.values[42] = "predict_cocina"
preds_y_datos_4.columns.values[43] = "p0_cocina"
preds_y_datos_4.columns.values[44] = "p1_cocina"

preds_y_datos_4.columns.values[45] = "predict_estufa"
preds_y_datos_4.columns.values[46] = "p0_estufa"
preds_y_datos_4.columns.values[47] = "p1_estufa"

preds_y_datos_4.columns.values[48] = "predict_parrilla"
preds_y_datos_4.columns.values[49] = "p0_parrilla"
preds_y_datos_4.columns.values[50] = "p1_parrilla"

In [22]:
preds_y_datos_4.columns

Index(['telefono', 'mobile', 'customerNoIfOffline', 'compras_domingo',
       'compras_lunes', 'compras_martes', 'compras_miercoles',
       'compras_jueves', 'compras_viernes', 'compras_sabado', 'compras_verano',
       'compras_otonio', 'compras_invierno', 'compras_primavera', 'comuna',
       'distintas_direcciones', 'depto_casa', 'edad', 'sexo',
       'cantidad_pedidos', 'suma_pedido', 'suma_descuentos', 'suma_kilos',
       'suma_p_CN02', 'suma_formato_05', 'suma_formato_11', 'suma_formato_15',
       'suma_formato_45', 'suma_p_CGH15', 'suma_p_CGH15_AL', 'suma_p_GGH15_AL',
       'frec_compra_promedio_dias', 'canal_offline', 'canal_online',
       'otro_canal', 'pago_efectivo', 'pago_deb_cred', 'pago_bco_estado',
       'pago_otros', 'predict_calefont', 'p0_calefont', 'p1_calefont',
       'predict_cocina', 'p0_cocina', 'p1_cocina', 'predict_estufa',
       'p0_estufa', 'p1_estufa', 'predict_parrilla', 'p0_parrilla',
       'p1_parrilla'],
      dtype='object')

In [23]:
preds_y_datos_4[preds_y_datos_4.telefono == 988176019]

,telefono,mobile,customerNoIfOffline,compras_domingo,compras_lunes,compras_martes,compras_miercoles,compras_jueves,compras_viernes,compras_sabado,...,p1_calefont,predict_cocina,p0_cocina,p1_cocina,predict_estufa,p0_estufa,p1_estufa,predict_parrilla,p0_parrilla,p1_parrilla


In [24]:
preds_y_datos_4[["telefono","predict_calefont","p0_calefont","p1_calefont","predict_cocina","p0_cocina","p1_cocina","predict_estufa","p0_estufa","p1_estufa","predict_parrilla","p0_parrilla","p1_parrilla"]]


,telefono,predict_calefont,p0_calefont,p1_calefont,predict_cocina,p0_cocina,p1_cocina,predict_estufa,p0_estufa,p1_estufa,predict_parrilla,p0_parrilla,p1_parrilla
0,NaN,1,0.484691,0.515309,1,0.326811,0.673189,0,0.941826,0.058174,0,0.985598,0.014402
1,10059328.0,1,0.433675,0.566325,1,0.252914,0.747086,1,0.713319,0.286681,0,0.981694,0.018306
2,225279038.0,1,0.314526,0.685474,1,0.141103,0.858897,0,0.865759,0.134241,0,0.986503,0.013497
3,225313432.0,1,0.309341,0.690659,1,0.144271,0.855729,0,0.869643,0.130357,0,0.973305,0.026695
4,225321546.0,1,0.374956,0.625044,1,0.187221,0.812779,0,0.873359,0.126641,0,0.992164,0.007836
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026998,942683801.0,0,0.573008,0.426992,0,0.901908,0.098092,0,0.774933,0.225067,0,0.992503,0.007497
1026999,967784894.0,0,0.496080,0.503920,1,0.378385,0.621615,1,0.681305,0.318695,0,0.986867,0.013133
1027000,224954482.0,1,0.301132,0.698868,1,0.224356,0.775644,0,0.831212,0.168788,0,0.981662,0.018338
1027001,987115161.0,1,0.386904,0.613096,1,0.347195,0.652805,1,0.723296,0.276704,0,0.989100,0.010900


In [25]:
preds_y_datos_4[["telefono","predict_calefont","p0_calefont","p1_calefont","predict_cocina","p0_cocina","p1_cocina","predict_estufa","p0_estufa","p1_estufa","predict_parrilla","p0_parrilla","p1_parrilla"]].to_gbq("gasco_modelos.20210218_prediccion_modelo_uso_gas", project_id=PID, if_exists="replace")




1it [00:55, 55.96s/it]
